In [2]:
import pandas as pd
import requests
import csv
import time
from bs4 import BeautifulSoup
from selenium import webdriver
import pickle

In [3]:
# Extract urls from website
def get_company_links():
        
    template = 'https://www.startbase.com/startups/?listOptions%5Bcompany-startup%5D=%7B%22version%22%3A1.3%2C%22sort%22%3A%22company.startbase_score%22%2C%22sortDirection%22%3A%22desc%22%2C%22display%22%3A%22small%22%2C%22itemsPerPage%22%3A4567%2C%22page%22%3A{}%2C%22userLocation%22%3Anull%2C%22filters%22%3A%7B%7D%7D'
    driver = webdriver.Firefox() # open browser

    links = []
    count = 1
    
    while True: # go through all urls
        try:
            url = template.format(count)
            driver.get(url) # go to url
            time.sleep(10) # wait for page to load
            element = driver.find_element_by_class_name('main-body') # get where links are            
            html = element.get_attribute('innerHTML') # obtain updated html with links
            
            soup = BeautifulSoup(html, 'html.parser')
            all_as = soup.find_all("a")            
            page_links = []
            for a in all_as:
                link = a.get("href")
                if link is not None and link.startswith('/organization/'):
                    page_links.append(link)           
            
            if not page_links: # when there are no more startups in the url
                driver.quit() # close browser
                break
            
            links = links + page_links
            count += 1    
            
        except:
            driver.quit()
            break

    return links

In [4]:
# Extract the info from a company url
def get_company_info(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Name, logo and teaser
    title_card = soup.find('div', 'flex-grow-1 mx-lg-4')
    startup_name = title_card.h1.get('data-name')
    logo = title_card.h1.get('data-icon')
    teaser = title_card.find('div', 'sb-teaser').text.strip()
    
    # Body info
    body_card = soup.find('ul', 'unformated row justify-content-start mb-0')
    info_cards = body_card.find_all('li')
    startup_info = []; labels_info = []

    for count, info in enumerate(info_cards):
        labels_info.append(info.text.strip().split(':')[0])
        try:
            if count != 9: #10th item is links to social media
                info_piece = info.text.strip()
                # text cleaning
            #         if type(info_piece) == str: # When empty
            #             info_piece = ''
                if '\n ' in info_piece:
                    info_piece = info_piece.split('\n ')[1]
                else:
                    info_piece = info_piece.split(': ')[1]

                if info_piece[0] == ' ':
                    info_piece = info_piece[1::]   
            else: #social media
                media_links = []
                for media in info.find_all('a'):
                    media_links.append(media.get('href'))
                info_piece = media_links
        except:
            info_piece = ''
        
        startup_info.append(info_piece)

    # About info
    try:
        about_company = soup.find('div', {'class':'sb-card mb-4'}, id='company').p.text
    except:
        about_company = ''

    # Team info
    try:
        # Get names
        member_names = soup.find_all('span',{'class':'name'})
        names = [];
        for name in member_names:
            names.append(name.text)
        # Get labels
        member_labels = soup.find_all('span',{'class':'label'})
        labels = [];
        for label in member_labels:
            labels.append(label.text) 

        team_members = dict(zip(names, labels))
    except:
        team_members = '';
    
    # Join extracted data
    info_data = [startup_name] + [logo] + [teaser] + startup_info + [about_company] + [team_members]
    info_labels = ['Name'] + ['Logo'] + ['Teaser'] + labels_info + ['About'] + ['Members']
    
    company_data = dict(zip(info_labels, info_data))
    
    return company_data

In [5]:
# Obtain all links of startups
links = get_company_links()
with open('links.pickle', 'wb') as f:
    pickle.dump(links, f)

In [42]:
# Get all data from links
#data = []

for i, link in enumerate(links):
    data.append(get_company_info('https://www.startbase.com' + link))
    if i % 50 == 0:
        print(i)     
dataframe = pd.DataFrame(data)
with open('startbasedata.pickle', 'wb') as f:
    pickle.dump(dataframe, f)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [49]:
# Convert to csv
dataframe.to_csv('startbase.csv', index=False)